## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-06-04-07-03 +0000,nyt,Hiroshima’s Pacifist Cause Is Losing Believers,https://www.nytimes.com/2025/08/06/world/asia/...
1,2025-08-06-04-01-09 +0000,nyt,How the Nuclear Attacks on Hiroshima and Nagas...,https://www.nytimes.com/2025/08/06/world/asia/...
2,2025-08-06-04-00-08 +0000,nyt,Chinese People Are Known to Be Big Savers. Man...,https://www.nytimes.com/2025/08/06/business/ch...
3,2025-08-06-04-00-00 +0000,wapo,Texas redistricting crisis has foreign echoes ...,https://www.washingtonpost.com/world/2025/08/0...
4,2025-08-06-03-52-09 +0000,nypost,Zohran Mamdani praises NYPD Commish Tisch’s cr...,https://nypost.com/2025/08/05/us-news/zohran-m...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2429/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
86,trump,47
44,new,13
98,gaza,11
191,tariffs,11
212,war,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
23,2025-08-06-01-44-41 +0000,nypost,Trump envoy Witkoff urged to take tough approa...,https://nypost.com/2025/08/05/us-news/trump-en...,86
98,2025-08-05-22-21-43 +0000,nypost,"Trump rips ‘no talent’ Gayle King, citing New ...",https://nypost.com/2025/08/05/media/trump-rips...,85
154,2025-08-05-19-42-59 +0000,nypost,Zelensky says Ukraine ready to ink drone deal ...,https://nypost.com/2025/08/05/us-news/zelensky...,84
35,2025-08-06-01-00-00 +0000,wsj,Federal Reserve governor Adriana Kugler is set...,https://www.wsj.com/economy/central-banking/tr...,80
86,2025-08-05-22-51-00 +0000,wsj,The Trump administration ran afoul of the Impo...,https://www.wsj.com/politics/policy/nih-fundin...,78


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
23,86,2025-08-06-01-44-41 +0000,nypost,Trump envoy Witkoff urged to take tough approa...,https://nypost.com/2025/08/05/us-news/trump-en...
257,51,2025-08-05-13-39-25 +0000,nypost,CEO ignored ‘red flags’ ahead of ‘preventable’...,https://nypost.com/2025/08/05/world-news/titan...
85,44,2025-08-05-22-53-49 +0000,nypost,Netanyahu believes occupying Gaza is the only ...,https://nypost.com/2025/08/05/world-news/netan...
46,43,2025-08-06-00-26-29 +0000,nyt,California Democrats Look to Redraw House Map ...,https://www.nytimes.com/2025/08/05/us/politics...
220,42,2025-08-05-16-34-00 +0000,wsj,The Swiss president is heading to the U.S. in ...,https://www.wsj.com/economy/trade/swiss-presid...
212,39,2025-08-05-16-53-34 +0000,nypost,Former X CEO Linda Yaccarino lands new gig at ...,https://nypost.com/2025/08/05/business/former-...
4,32,2025-08-06-03-52-09 +0000,nypost,Zohran Mamdani praises NYPD Commish Tisch’s cr...,https://nypost.com/2025/08/05/us-news/zohran-m...
224,32,2025-08-05-16-15-00 +0000,wsj,The way Illinois draws political district boun...,https://www.wsj.com/politics/policy/pritzkers-...
154,32,2025-08-05-19-42-59 +0000,nypost,Zelensky says Ukraine ready to ink drone deal ...,https://nypost.com/2025/08/05/us-news/zelensky...
45,31,2025-08-06-00-31-37 +0000,nypost,Ex-Miami Heat security worker stole and sold $...,https://nypost.com/2025/08/05/sports/retried-m...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
